In [ ]:
# !cp /content/drive/MyDrive/images.zip /content/images.zip

In [ ]:
# !7za x /content/images.zip

In [ ]:
# !cp /content/drive/MyDrive/annotations.json /content/annotations.json

In [ ]:
!pip install torchmetrics

In [ ]:
import os
import json
import gc

from pathlib import Path
from PIL import Image
from PIL import Image, ImageFile
# Allow loading truncated images; still catch severe cases
ImageFile.LOAD_TRUNCATED_IMAGES = True

import torch
from torch.utils.data import Dataset
from transformers import (
    AutoImageProcessor,
    DeformableDetrForObjectDetection,
    TrainingArguments,
    Trainer
)

# 1) Load HF processor & model

def get_hf_model_and_processor(num_classes):
    processor = AutoImageProcessor.from_pretrained(
        "SenseTime/deformable-detr",
        size={"height": 256, "width": 256},
        do_resize=True,      # (the default) actually perform the resize
        resample=2
    )
    model = DeformableDetrForObjectDetection.from_pretrained(
        "SenseTime/deformable-detr",
        num_labels=num_classes,
        ignore_mismatched_sizes=True
    )
    return processor, model

In [ ]:
# 2) Custom PyTorch Dataset storing raw COCO-format annotations
class ObjectDetectionDataset(Dataset):
    def __init__(self, images_dir, annotations_file):
        self.root = Path(images_dir)
        with open(annotations_file) as f:
            data = json.load(f)

        # Map image_id to path and store raw annotations
        self.image_paths = {}
        self.raw_annotations = {}
        for img in data['images']:
            fid = img['id']
            fname = img.get('file_name', f"{fid}.jpg")
            path = self.root / fname
            if path.exists():
                self.image_paths[fid] = path
                self.raw_annotations[fid] = []
        for ann in data['annotations']:
            fid = ann['image_id']
            if fid in self.image_paths:
                x,y,w,h = ann['bbox']
                if w>0 and h>0:
                    self.raw_annotations[fid].append({
                        'bbox': [x, y, w, h],
                        'category_id': ann['category_id'],
                        'area': w * h,
                        'iscrowd': ann.get('iscrowd', 0)
                    })
        self.image_ids = list(self.image_paths.keys())
        if not self.image_ids:
            raise ValueError(f"No images found in {images_dir}")
        del data; gc.collect()

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        fid = self.image_ids[idx]
        image = Image.open(self.image_paths[fid]).convert("RGB")
        anns = self.raw_annotations.get(fid, [])
        # Return PIL image and raw COCO annotations
        return image, {'image_id': fid, 'annotations': anns}

In [ ]:
# 3) HF Dataset wrapper using processor to handle annotation conversion
class HFDataset(Dataset):
    def __init__(self, base_dataset, processor):
        self.base = base_dataset
        self.processor = processor

    def __len__(self):
        return len(self.base)

    def __getitem__(self, idx):
        # Load raw image and COCO-format annotations
        image, coco_target = self.base[idx]
        # Let the processor resize image and convert COCO-format bboxes [x,y,w,h]
        # into normalized [x0,y0,x1,y1] for the model
        encoded = self.processor(
            images=image,
            annotations=coco_target,
            return_tensors="pt"
        )
        pixel_values = encoded.pixel_values.squeeze(0)  # [3,H,W]
        labels = encoded.labels[0]  # {'boxes': Tensor[N,4], 'class_labels': Tensor[N]}
        return {'pixel_values': pixel_values, 'labels': labels}

In [ ]:
# 4) Data collator: batch pixel_values and labels list) Data collator) Data collator: batch pixel_values and labels list

def collate_fn(batch):
    pixel_values = torch.stack([item['pixel_values'] for item in batch])
    labels = [item['labels'] for item in batch]
    return {'pixel_values': pixel_values, 'labels': labels}

In [ ]:
# 5) Compute mAP with torchmetrics
import random
from torch.utils.data import Subset
from torchmetrics.detection.mean_ap import MeanAveragePrecision

# def compute_metrics(eval_pred):
#     predictions, references = eval_pred
#     # predictions: list of dicts with 'boxes','scores','labels'
#     # references: list of dicts with 'boxes','labels'
#     metric = MeanAveragePrecision(box_format="xyxy")
#     # accumulate batches
#     for preds, refs in zip(predictions, references):
#         # preds is dict, refs is dict
#         # print(preds)
#         # print(refs)
#         # print(1/0)
#         metric.update(
#             [{'boxes': preds['boxes'], 'scores': preds['scores'], 'labels': preds['labels']}],
#             [{'boxes': refs['boxes'], 'labels': refs['class_labels']}]  # target labels use 'class_labels'
#         )
#     return metric.compute()

In [ ]:
import torch
from torchmetrics.detection.mean_ap import MeanAveragePrecision

# # Instantiate once (so the class list is remembered across eval calls)
# metric = MeanAveragePrecision(box_format="xyxy", class_metrics=True)

# def compute_metrics(eval_pred):
#     """
#     eval_pred.predictions: tuple of two numpy arrays:
#         - scores:    shape (batch, num_queries, num_classes+1)
#         - pred_boxes: shape (batch, num_queries, 4), normalized [0,1]
#     eval_pred.label_ids: list of dicts, each with:
#         - 'boxes': tensor [num_objects,4] normalized [0,1]
#         - 'class_labels': tensor [num_objects]
#     """
#     # print(eval_pred)
#     scores_np, boxes_np = eval_pred.predictions
#     labels_list        = eval_pred.label_ids

#     # Reset metric state at each evaluation
#     metric.reset()

#     batch_size = scores_np.shape[0]

#     # Loop over batch
#     for i in range(batch_size):
#         # 1) Extract raw model outputs for this image
#         scores    = torch.from_numpy(scores_np[i])    # [Q, C+1]
#         boxes     = torch.from_numpy(boxes_np[i])     # [Q, 4]

#         # 2) Remove the “no-object” class and pick per-query best label
#         pred_scores = torch.softmax(scores[:, :-1], dim=-1)      # [Q, C]
#         pred_labels = torch.argmax(pred_scores, dim=-1)          # [Q]
#         pred_scores = pred_scores[torch.arange(pred_scores.size(0)), pred_labels]  # [Q]

#         # 3) Gather only the positive boxes (you may want to threshold by score)
#         keep = pred_scores > 0.05
#         filtered_boxes  = boxes[keep]
#         filtered_scores = pred_scores[keep]
#         filtered_labels = pred_labels[keep]

#         # 4) Build the one-element batch for torchmetrics
#         metric.update(
#             preds=[{
#                 "boxes": filtered_boxes,
#                 "scores": filtered_scores,
#                 "labels": filtered_labels
#             }],
#             target=[{
#                 "boxes": labels_list[i]["boxes"].to(filtered_boxes.dtype),
#                 "labels": labels_list[i]["class_labels"]
#             }]
#         )

#     # 5) Compute and return the aggregated mAP metrics
#     result = metric.compute()  # returns dict with 'map', 'map_50', 'map_75', and per-class metrics
#     # Convert all Tensors to floats for logging
#     return {k: float(v) for k,v in result.items()}

In [ ]:
script_dir = "/content"
# Corrected images directory path:
images_dir = os.path.join(script_dir, "novice", "cv", "images")
ann_file    = os.path.join(script_dir, "annotations.json")
num_classes=18
output_dir="./detr-finetuned"
train_bs=32
eval_bs=48
epochs=20
lr=5e-5
save_steps=500

# load processor and model
processor, model = get_hf_model_and_processor(num_classes)

# Shuffle and split dataset
base = ObjectDetectionDataset(images_dir, ann_file)
indices = list(range(len(base)))
random.shuffle(indices)
split = int(0.8 * len(indices))
train_idx, eval_idx = indices[:split], indices[split:]
train_base = torch.utils.data.Subset(base, train_idx)
eval_base = torch.utils.data.Subset(base, eval_idx)

train_ds = HFDataset(train_base, processor)
eval_ds = HFDataset(eval_base, processor)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for conv1.weight: copying from a non-meta parameter in the checkpoint to

In [ ]:
from tqdm import tqdm

In [ ]:
# for i in tqdm(range(len(train_ds))):
#     try:
#         image = train_ds[i]['pixel_values']
#     except:
#         print(i)
#         continue
#     boxes1 = train_ds[i]['labels']['boxes']
#     assert (boxes1[:, 2:] >= boxes1[:, :2]).all()

In [ ]:
# training args
args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=56,
    per_device_eval_batch_size=eval_bs,
    num_train_epochs=epochs,
    learning_rate=5e-5,
    weight_decay=1e-4,
    eval_strategy="steps",
    save_steps=save_steps,
    eval_steps=50,
    logging_dir=os.path.join(output_dir, "logs"),
    logging_steps=max(1, save_steps // 5),
    # fp16=torch.cuda.is_available(),
    bf16=True,
    # fp16_opt_level="O3",
    report_to="none",
    remove_unused_columns=False,
)

# trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=collate_fn,
    tokenizer=processor,
    # compute_metrics=compute_metrics
)

<ipython-input-120-3c83fb300e53>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
50,No log,0.615693
100,0.533000,0.608230
150,0.533000,0.595276
200,0.507100,0.607277


KeyboardInterrupt: 

In [ ]:
trainer.save_model()

In [ ]:
# 3) Instantiate the torchmetrics mAP
metric = MeanAveragePrecision(box_format="xyxy", class_metrics=False)

# 4) Run inference & update metric
cnt = 0
with torch.no_grad():
    for batch in tqdm(eval_ds):
        cnt += 1
        # if cnt >= 100:
        #     break
        # `collate_fn` returns { pixel_values: Tensor[B,3,H,W], labels: List[GT_dict] }
        pixel_values = batch["pixel_values"].to("cuda")
        gt_labels = batch["labels"]  # list of dicts, each with 'boxes' & 'class_labels'

        # forward
        outputs = model(pixel_values=pixel_values.unsqueeze(0))

        # the model returns list of dicts: { 'logits', 'pred_boxes', ... }
        # call post_process to get final predictions in xyxy pixel coords:
        # target_sizes = torch.stack([torch.tensor([d["orig_size"][1], d["orig_size"][0]])
        #                             for d in gt_labels]).to("cuda")
        # results = processor.post_process_object_detection(
        #     outputs, target_sizes=target_sizes, threshold=0.5
        # )

        # Prepare lists for metric.update
        # print(outputs['pred_boxes'])
        # print(outputs['logits'].shape)
        # print(gt_labels)
        preds = []
        targets = []
        bbox = outputs['pred_boxes'].cpu().squeeze()
        bbox[:, [2, 3]] = bbox[:, [2, 3]] + bbox[:, [0, 1]]
        scores = torch.max(torch.softmax(outputs['logits'], dim=-1), dim=-1)[0]
        labels = torch.argmax(outputs['logits'], dim=-1)
        preds.append({
            "boxes":  bbox,
            "scores": scores.cpu().squeeze(),
            "labels": labels.cpu().squeeze()
        })
        # print(bbox[0])
        # print(scores.cpu().shape)
        # print(labels.cpu().shape)
        # ground truth
        gt_labels["boxes"] = gt_labels["boxes"].cpu()
        gt_labels["boxes"][:, [2, 3]] = gt_labels["boxes"][:, [2, 3]] + gt_labels["boxes"][:, [0, 1]]
        targets.append({
            "boxes":      gt_labels["boxes"].cpu(),
            "labels":     gt_labels["class_labels"].cpu()
        })
        # print(targets)
        # break

        # update metric
        metric.update(preds, targets)

# 5) Compute & print
metrics = metric.compute()
print("==== Evaluation results ====")
print(f"mAP (0.50:0.95): {metrics['map']:.4f}")
print(f"mAP@0.50:    {metrics['map_50']:.4f}")
print(f"mAP@0.75:    {metrics['map_75']:.4f}")

  0%|          | 0/671 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: Encountered more than 100 detections in a single image. This means that certain detections with the lowest scores will be ignored, that may have an undesirable impact on performance. Please consider adjusting the `max_detection_threshold` to suit your use case. To disable this warning, set attribute class `warn_on_many_detections=False`, after initializing the metric.
  warnings.warn(*args, **kwargs)
100%|██████████| 671/671 [01:04<00:00, 10.41it/s]


==== Evaluation results ====
mAP (0.50:0.95): 0.3579
mAP@0.50:    0.4602
mAP@0.75:    0.4088


In [ ]:
metrics

{'map': tensor(0.3579),
 'map_50': tensor(0.4602),
 'map_75': tensor(0.4088),
 'map_small': tensor(0.3579),
 'map_medium': tensor(-1.),
 'map_large': tensor(-1.),
 'mar_1': tensor(0.3771),
 'mar_10': tensor(0.4588),
 'mar_100': tensor(0.4613),
 'mar_small': tensor(0.4613),
 'mar_medium': tensor(-1.),
 'mar_large': tensor(-1.),
 'map_per_class': tensor(-1.),
 'mar_100_per_class': tensor(-1.),
 'classes': tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17],
        dtype=torch.int32)}

In [ ]:
# from google.colab import files
# files.download('/content/detr-finetuned/model.safetensors')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!cp /content/detr-finetuned/model.safetensors /content/drive/MyDrive/detr-finetuned.safetensors